# 사진 자동 정리하기

사진 정보를 이용하여 위치, 날짜등의 이름으로 사진의 이름을 변경하여 정리합니다. 

[정리 전]

KakaoTalk_2022-05-06 14-58-50.jpg


[정리 후]

시흥시_은행동_2022-06-04 14-06-00.jpg


---

개발 프로세스

1. 사진파일에서 위도, 경도(GPS 정보 추출) 추출
2. 위도, 경도 -> 주소로 변환
3. 파일이름 변경

----

## 사진 정보 추출하기

In [6]:
from PIL import Image

image = Image.open(r'사진/시흥시_은행동_2022-06-04 14-06-00.jpg')
info = image._getexif();
info

{34853: {0: b'\x02\x02\x00\x00',
  2: (37.0, 26.0, 41.294399),
  4: (126.0, 47.0, 53.931119),
  6: 52.0},
 296: 2,
 34665: 262,
 271: 'samsung',
 272: 'SM-S908N',
 305: 'S908NKSU1AVE4',
 306: '2022:06:04 14:06:00',
 531: 2,
 532: (0.0, 255.0, 128.0, 255.0, 128.0, 255.0),
 282: 150.0,
 283: 150.0,
 36864: b'0221',
 37121: b'\x01\x02\x03\x00',
 37377: 7.55,
 36867: '2022:06:04 14:06:00',
 36868: '2022:06:04 14:06:00',
 37378: 2.27,
 37379: 5.96,
 37380: 0.0,
 37381: 2.27,
 37383: 2,
 37385: 0,
 37386: 3.8,
 40961: 1,
 40962: 841,
 41989: 25,
 41990: 0,
 40963: 1121,
 33434: 0.005319148936170213,
 33437: 2.2,
 34850: 2,
 34855: 50,
 41986: 0,
 40960: b'0100',
 41987: 0,
 41988: 1.23}

In [9]:
# 파일 속성정보 출력
from PIL.ExifTags import TAGS
for tag,value in info.items():
    decoded = TAGS.get(tag, tag)
    print(TAGS.get(tag, tag))

GPSInfo
ResolutionUnit
ExifOffset
Make
Model
Software
DateTime
YCbCrPositioning
ReferenceBlackWhite
XResolution
YResolution
ExifVersion
ComponentsConfiguration
ShutterSpeedValue
DateTimeOriginal
DateTimeDigitized
ApertureValue
BrightnessValue
ExposureBiasValue
MaxApertureValue
MeteringMode
Flash
FocalLength
ColorSpace
ExifImageWidth
FocalLengthIn35mmFilm
SceneCaptureType
ExifImageHeight
ExposureTime
FNumber
ExposureProgram
ISOSpeedRatings
ExposureMode
FlashPixVersion
WhiteBalance
DigitalZoomRatio


In [15]:
from PIL import Image
from PIL.ExifTags import TAGS

image = Image.open(r'사진/시흥시_은행동_2022-06-04 14-06-00.jpg')
info = image._getexif();

# 속성정보
gps_lat = None
gps_lon = None
for tag, value in info.items():
    decoded = TAGS.get(tag, tag)
    # print(f'{decoded} : {value}')
    if decoded == 'GPSInfo':
        print(f'{decoded} : {value}')

GPSInfo : {0: b'\x02\x02\x00\x00', 2: (37.0, 26.0, 41.294399), 4: (126.0, 47.0, 53.931119), 6: 52.0}


In [24]:
from PIL import Image
from PIL.ExifTags import TAGS

image = Image.open(r'사진/시흥시_은행동_2022-06-04 14-06-00.jpg')
info = image._getexif();

# 속성정보
gps_lat = None
gps_lon = None
for tag,value in info.items():
    decoded = TAGS.get(tag,tag)
    # print(f'{decoded} : {value}')
    if decoded == 'GPSInfo':
        gps_lat = value.get(2)
        gps_lon = value.get(4)

print(gps_lat, gps_lon)

(37.0, 26.0, 41.294399) (126.0, 47.0, 53.931119)


In [28]:
# 위도 경도 값 보정

lat = (((gps_lat[2] / 60.0) + gps_lat[1]) / 60.0) + gps_lat[0]
lon = (((gps_lon[2] / 60.0 ) +  gps_lon[1]) / 60.0) + gps_lon[0]

In [30]:
lat, lon

(37.444803999722225, 126.79831419972223)

## 위도,경도에서 주소로 변환

https://pypi.org/project/geopy/

In [31]:
%pip install geopy

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 KB 5.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [41]:
# 지오코드 사용법 살펴보기
# 주소를 위도 경도로 변환

from geopy.geocoders import Nominatim

test = '성북구 오패산로 46'
geolocator = Nominatim(user_agent='South Korea') # user_agent는 식별가능 문자열(아무 문자열)
location = geolocator.geocode(test)

latitude = location.latitude #위도
longitude = location.longitude #경도

print(latitude, longitude)

37.6070327 127.0383656


In [42]:
# 위도 경도를 주소로 변환
lat_lng_str = f'{latitude},{longitude}' # 위도 경도 문자열
loc = geolocator.reverse(lat_lng_str)
loc.address

'꼬꼬스토리, 46, 주상가동 136호 (하월곡동), 오패산로, 월곡1동, 성북구, 서울특별시, 02741, 대한민국'

---

## 함수 생성

In [47]:
from PIL import Image
from PIL.ExifTags import TAGS
# 사진 정보 가져오기
def getPicInfo(fname):
    image = Image.open(fname)
    info = image._getexif();
    image.close() # close()를 해줄 것. 파일이름 변경시 에러발생 가능성 있음

    # 속성정보
    lat = None
    lone = None
    pictime = None

    for tag,value in info.items():
        decoded = TAGS.get(tag, tag)
        # print(f'{decoded} : {value}')
        if decoded == "GPSInfo":
            gps_lat = value.get(2)
            gps_lon = value.get(4)

            # 보정
            lat = (((gps_lat[2] / 60.0 ) +  gps_lat[1]) / 60.0) + gps_lat[0]
            lon = (((gps_lon[2] / 60.0 ) +  gps_lon[1]) / 60.0) + gps_lon[0]
        
        if decoded == "DateTime":
            pictime = value.replace(":","-")

    return lat,lon,pictime

getPicInfo(r'사진/시흥시_은행동_2022-06-04 14-06-00.jpg')

(37.444803999722225, 126.79831419972223, '2022-06-04 14-06-00')

In [49]:
from geopy.geocoders import Nominatim

# 주소 가져오기
def getAddress(latitude, longitude):

    lat_lng_str = f'{latitude},{longitude}' # 위도 경도 문자열

    geolocator = Nominatim(user_agent='South Korea')

    loc = geolocator.reverse(lat_lng_str)
    addlist = loc.address.split(",")
    si = addlist[3].strip()
    do = addlist[2].strip()
    return si, do

getAddress(37.444803999722225,126.79831419972223)

('은행동', '은행로')

## 종합코드

In [51]:
from glob import glob
import os

flist = glob(r'사진\*.jpg')

for fname in flist:

    lat, lon, pictime = getPicInfo(fname)
    si,do = getAddress(lat,lon)
    # print(lat, lon, pictime)
    # print(si,do)
    newfname = f'사진\\{si}_{do}_{pictime}.jpg'
    # print(newfname)

    os.rename(fname,newfname)

## 에러발생하는 경우 시뮬레이션

In [54]:
from PIL import Image
from PIL.ExifTags import TAGS
pic_time = f'./사진/KakaoTalk_20250109_001405527_01.jpg'
image = Image.open(pic_time) # 파일 열기
info = image._getexif()

In [55]:
os.rename(pic_time,'./사진/aaa.jpg') #이름 변경 가능

In [ ]:
# 최종
from PIL import Image
from PIL.ExifTags import TAGS

pic_time = f'./사진/KakaoTalk_20250109_001405527_01.jpg'

try:
    image = Image.open(pic_time) #파일을 오픈하고 비정상적으로 종료시 파일스트림이 유지됨
    info = image._getexif() 

    raise Exception("내가발생시킨 에러")


except Exception as e: 
    print(e)
    # image.close()
    import os
    os.rename(pic_time,'./사진/aaa.jpg')

내가발생시킨 에러
